<a href="https://colab.research.google.com/github/dswh/lil_nlp_with_tensorflow/blob/main/02_02_begin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings for Sentiment Analysis

This notebook explains how to add the embeddings layer to the neural network. We will train our own word embeddings using a simple Keras model for a sentiment classification task.

Steps include:
1. Downloading data from tensorflow dataset.
2. Segregating training and testing sentences & labels.
3. Data preparation to padded sequences
4. Defining out Keras model with an Embedding layer.
5. Train the model and explore the weights from the embedding layer.


In [2]:
##import the required libraries and APIs
import numpy as np
import tensorflow as tf
!pip install tensorflow_datasets
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

     |████████████████████████████████| 4.3 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 112 kB/s eta 0:00:01
     |████████████████████████████████| 211 kB 1.5 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21485 sha256=849874303f035f24985ac5ff021eec53bd37ab4b2c445a8bdb36f92a290dc3c1
  Stored in directory: /root/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
2.9.1


## Downloading the TensorFlow `imdb_review` dataset

> Make sure tensorflow_datasets is installed

In [6]:
##load the imdb reviews dataset
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLG55E8/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLG55E8/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLG55E8/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Segregating training and testing sets

In [7]:
##segregate training and test set
train_data, test_data = data['train'], data['test']

##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [8]:
##iterate over the train data to extract sentences and labels
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

##iterate over the test set to extract sentences and labels
for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())



In [9]:
##convert lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Data preparation - setting up the tokenizer

In [11]:
##define the parameters for the tokenizing and padding
vocab_size = 1000
embedding_dim = 16
max_length = 150
trunc_type='post'
oov_tok = "<oov>"


In [13]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen=max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length)

In [14]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0  11  26  75 571   6 805   1 313 106  19  12   7 629 686   6   4
   1   5 181 584  64   1 110   1   3   1  21   2   1   3 258  41   1   4
 174 188  21  12   1  11   1   1  86   2  20  14   1   2 112 940  14   1
   1 548   3 355 

## Define the Neural Network with Embedding layer

1. Use the Sequential API.
2. Add an embedding input layer of input size equal to vocabulary size.
3. Add a flatten layer, and two dense layers.

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'), # for classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])

##compile the model with loss function, optimizer and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           16000     
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 6)                 14406     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 30,413
Trainable params: 30,413
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [16]:
num_epochs = 10

##train the model with training and validation set
model.fit(
    train_padded, 
    train_labels, 
    epochs=num_epochs, 
    validation_data=(test_padded, test_labels)
    )

Epoch 1/10
782/782 [==============================] - 10s 12ms/step - loss: 0.5193 - accuracy: 0.7256 - val_loss: 0.3636 - val_accuracy: 0.8389
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3689 - accuracy: 0.8370 - val_loss: 0.3694 - val_accuracy: 0.8355
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3170 - accuracy: 0.8680 - val_loss: 0.3785 - val_accuracy: 0.8290
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2558 - accuracy: 0.8994 - val_loss: 0.4251 - val_accuracy: 0.8166
Epoch 5/10
782/782 [==============================] - 7s 10ms/step - loss: 0.1968 - accuracy: 0.9282 - val_loss: 0.4820 - val_accuracy: 0.8088
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1433 - accuracy: 0.9534 - val_loss: 0.5551 - val_accuracy: 0.8015
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1012 - accuracy: 0.9699 - val_loss: 0.6548 - val_accuracy: 0.792

## Deriving weights from the embedding layer

In [18]:
##isolating the first embedding layer
l1 = model.layers[0]

##extracting learned weights
weights = l1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights)

(1000, 16)
[[ 0.01653522 -0.00327209 -0.05449872 ... -0.01003235 -0.00121851
  -0.01222734]
 [ 0.03183649  0.01428665 -0.02701611 ... -0.01070418  0.02977063
   0.01566753]
 [ 0.01609767 -0.0218314  -0.05049689 ... -0.03955908  0.04542179
  -0.03035884]
 ...
 [-0.1315037   0.21381466  0.12779288 ... -0.10847312 -0.17082211
  -0.22325724]
 [-0.01705533  0.02146531 -0.15523589 ... -0.21621649  0.04260562
  -0.19332454]
 [-0.02548479  0.08473054  0.2005548  ...  0.0978324   0.01761167
   0.31239313]]
